In [1]:
import pandas as pd
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [2]:
# === 1. Load CSV data ===
train = pd.read_csv("X_train.csv")      # must contain 'Close'
test = pd.read_csv("X_test.csv")      # missing 'Close'

In [3]:
# === 2. Prepare data ===
features = ['Open', 'High', 'Low', 'Volume']
train = train.dropna(subset=['Close'])
X = train[features].values
y = train['Close'].values

In [4]:
# Normalize input features and targets
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

In [5]:
# === 3. Quantum device ===
n_qubits = len(features)
dev = qml.device("default.qubit", wires=n_qubits)

In [6]:
# === 4. Define variational circuit ===
def circuit(params, x):
    qml.AngleEmbedding(x, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev)
def qnode(params, x):
    return circuit(params, x)

In [7]:
# === 5. Define quantum regression model ===
class QuantumRegressor:
    def __init__(self, n_layers=2):
        self.n_layers = n_layers
        self.params = pnp.random.uniform(low=0, high=np.pi, size=(n_layers, n_qubits, 3))

    def predict(self, X):
        preds = []
        for x in X:
            result = qnode(self.params, x)
            preds.append(float(result))  # convert ArrayBox -> float
        return np.array(preds)

    def fit(self, X, y, lr=0.1, epochs=50):
        opt = qml.GradientDescentOptimizer(stepsize=lr)
        for epoch in range(epochs):
            self.params, loss = self._step(opt, X, y)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: loss = {loss:.4f}")

    def _step(self, opt, X, y):
        def cost(params):
            preds = pnp.array([qnode(params, x) for x in X])
            return pnp.mean((preds - y) ** 2)
        new_params = opt.step(cost, self.params)
        loss = cost(self.params)
        return new_params, float(loss)  # cast loss to float for printing

In [8]:
# === 6. Train the model ===
qml_model = QuantumRegressor()
qml_model.fit(X_scaled, y_scaled, lr=0.2, epochs=60)

Epoch 0: loss = 0.5844
Epoch 10: loss = 0.2403
Epoch 20: loss = 0.0495
Epoch 30: loss = 0.0331
Epoch 40: loss = 0.0253
Epoch 50: loss = 0.0198


In [9]:
# === 7. Evaluate ===
y_train_pred_scaled = qml_model.predict(X_scaled)
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()

mse = mean_squared_error(train['Close'], y_train_pred)
r2 = r2_score(train['Close'], y_train_pred)
print(f"\n🔍 Training MSE: {mse:.4f}")
print(f"🔍 Training R²: {r2:.4f}")


🔍 Training MSE: 36305.6962
🔍 Training R²: 0.6686


In [10]:
# === 8. Predict missing Close values in test set ===
X_test = test[features].values
X_test_scaled = scaler_X.transform(X_test)
y_pred_scaled = qml_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()




In [11]:
# === 9. Save results ===
test['Predicted_Close_Val'] = y_pred
test.to_csv("predictions.csv", index=False)

print("✅ Quantum prediction complete! Results saved to 'prediction.csv'")

✅ Quantum prediction complete! Results saved to 'prediction.csv'
